In [154]:
import rasterio
import geopandas as gpd
import glob
import zipfile

In [155]:
gdf_scms = gpd.read_file('data/folhas-20-oumais-quadriculas-treinamento.gpkg').to_crs(5533)

In [156]:
gdf_quadriculas = gpd.read_file('data/quadriculas-32m-nas-areas-de-favela-atual.gpkg', layer='quadriculas-de-treinamento').to_crs(5533)

In [149]:
gdf_edificacoes = gpd.read_file('data/edificacoes-nas-quadriculas.gpkg')

In [157]:
gdf_scm_quadriculas = gpd.sjoin(gdf_scms, gdf_quadriculas, how='right')

In [151]:
gpd.sjoin(gdf_quadriculas, gdf_edificacoes, how='right')

<ipython-input-151-688c547a3b62>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31983
Right CRS: EPSG:5533

  gpd.sjoin(gdf_quadriculas, gdf_edificacoes, how='right')


,index_left,left,top,right,bottom,id,CLASSID,FEATID,DATA_CRIAC,ESCALA,ALTURA_EDI,COD_SETOR_,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4875.0,2007-03-11,1:1000,5.632,034,"MULTIPOLYGON (((334195.394 7392885.796, 334197..."
1,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4876.0,2007-03-11,1:1000,5.532,034,"MULTIPOLYGON (((334198.192 7392873.669, 334193..."
2,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4878.0,2007-03-11,1:1000,7.072,034,"MULTIPOLYGON (((334181.198 7392874.087, 334177..."
3,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4879.0,2007-03-11,1:1000,7.392,034,"MULTIPOLYGON (((334177.987 7392867.815, 334183..."
4,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4675.0,2007-03-11,1:1000,7.190,034,"MULTIPOLYGON (((334165.920 7392841.071, 334162..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90031,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2810892.0,2007-03-11,1:1.000,2.890,062,"MULTIPOLYGON (((340932.652 7397339.384, 340941..."
90032,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815655.0,2007-03-11,1:1000,4.620,033,"MULTIPOLYGON (((334015.626 7392801.673, 333996..."
90033,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815661.0,2007-03-11,1:1000,7.750,033,"MULTIPOLYGON (((334088.563 7392770.539, 334052..."
90034,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815662.0,2007-03-11,1:1000,3.119,033,"MULTIPOLYGON (((334029.463 7392827.406, 334029..."


In [192]:
from rasterio.windows import Window
from rasterio.transform import Affine
import math

In [214]:
folder = 'ortofotosMDC2004/treinamento'

# Interando sobre cada folha SCM
for scm in gdf_scms.qo2004_cod.to_list():
    
    # DEscomprime o ZIP correspondente a folha SCM
    with zipfile.ZipFile(f'{folder}/1O_{scm}.zip', 'r') as zip_ref:
        zip_ref.extractall(folder)

    # Abre o raster
    dataset = rasterio.open(f'{folder}/1O_{scm}_060803.JPG')

    print(dataset.profile)

    # Intera sobre cada quadricula da folha SCM Atual
    for row in gdf_scm_quadriculas.loc[gdf_scm_quadriculas.qo2004_cod == scm].itertuples():
        
        # Janela de 256x256
        xoff = math.floor((row.geometry.bounds[0] - dataset.bounds.left))/.125
        yoff = math.floor((row.geometry.bounds[1] - dataset.bounds.bottom))/.125
        
        window = Window(xoff, yoff, 256, 256)
        # transform = dataset.window_transform(window)
        transform = Affine.translation((xoff * 0.125) + dataset.bounds.left, dataset.bounds.bottom + (yoff * 0.125)) * Affine.scale(.125, -.125)
        # transform = Affine(.125, 0., (xoff * 0.125) + dataset.bounds.left, 0., -.125, dataset.bounds.bottom + (yoff * 0.125))

        profile = dataset.profile
        profile.update({
            'height': 256,
            'width': 256,
            'crs':'EPSG:5533',
            'transform': transform})

        print(transform)

        with rasterio.open(f'ortofotosMDC2004/licoes/{scm}-{row.Index}.tif', 'w', **profile) as dst:
            # Read the data from the window and write it to the output raster
            dst.write(dataset.read(window=window))
        # Edificacoes na determinada janela
        

    break

{'driver': 'JPEG', 'dtype': 'uint8', 'nodata': None, 'width': 4060, 'height': 4748, 'count': 3, 'crs': None, 'transform': Affine(0.125, 0.0, 316770.574,
       0.0, -0.125, 7382679.79), 'tiled': False, 'compress': 'jpeg', 'interleave': 'pixel', 'photometric': 'ycbcr'}
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382430.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382400.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382250.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382220.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382190.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382160.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382370.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382310.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317068.57|
| 0.00,-0.12, 7382280.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317038.57|
| 0.00,-0.12, 7382220.29|
| 0.00, 0.00, 1.00|
| 0.12, 0.00, 317038.

In [180]:
abs(-1234)

1234